In [2]:
import numpy as np # library to handle data in a vectorized manner
import pandas as pd # library for data analsysis

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

!conda install -c conda-forge beautifulsoup4 --yes

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.11

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.

Libraries imported.


<b>Step 1: After importing all the required libraries, we will perform a process called Web Scraping. In this process we will download the HTML data from the webpage under observation and extract the data from this page using the BeautifulSoup library.</b>

In [3]:
#Downloading HTML contents of the given web page
webpage = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M") #requesting the web server for HTML contents of a given web page
print("The status code is ",webpage.status_code) # if the status code is 200 then the web page is successfully downloaded
if (webpage.status_code == 200):
    print("The webpage is successfully downloaded")
else:
    print("Failed to download the webpage")
#print("HTML content of the Web page without parsing")
#webpage.content

The status code is  200
The webpage is successfully downloaded


In [4]:
#parsing the page and extracting the cotnents from p tag 
from bs4 import BeautifulSoup
soup = BeautifulSoup(webpage.content, 'html.parser')
#print(soup.prettify())

In [5]:
#soup.find_all(class_= "wikitable sortable") #extracting table from the web page contents
table = soup.find(class_="wikitable sortable")
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [6]:
rows = soup.find_all('tr')
print(rows[:10])

[<tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>, <tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>, <tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>, <tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>, <tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>, <tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>, <tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>, <tr>
<td>M6A</td>
<td><a href="/wiki/North_York" ti

<b>Step2 : We will now transfer the data from the HTML table into a lsit</b>

In [8]:
#extracting the contents from the HTML table into a list
table_text=[]
for tr in table.findAll('tr'):
    tds = tr.findAll('td')
    for td in tds:
        table_text.append(td.get_text())    
table_text

['M1A',
 'Not assigned',
 'Not assigned\n',
 'M2A',
 'Not assigned',
 'Not assigned\n',
 'M3A',
 'North York',
 'Parkwoods\n',
 'M4A',
 'North York',
 'Victoria Village\n',
 'M5A',
 'Downtown Toronto',
 'Harbourfront\n',
 'M5A',
 'Downtown Toronto',
 'Regent Park\n',
 'M6A',
 'North York',
 'Lawrence Heights\n',
 'M6A',
 'North York',
 'Lawrence Manor\n',
 'M7A',
 "Queen's Park",
 'Not assigned\n',
 'M8A',
 'Not assigned',
 'Not assigned\n',
 'M9A',
 'Etobicoke',
 'Islington Avenue\n',
 'M1B',
 'Scarborough',
 'Rouge\n',
 'M1B',
 'Scarborough',
 'Malvern\n',
 'M2B',
 'Not assigned',
 'Not assigned\n',
 'M3B',
 'North York',
 'Don Mills North\n',
 'M4B',
 'East York',
 'Woodbine Gardens\n',
 'M4B',
 'East York',
 'Parkview Hill\n',
 'M5B',
 'Downtown Toronto',
 'Ryerson\n',
 'M5B',
 'Downtown Toronto',
 'Garden District\n',
 'M6B',
 'North York',
 'Glencairn\n',
 'M7B',
 'Not assigned',
 'Not assigned\n',
 'M8B',
 'Not assigned',
 'Not assigned\n',
 'M9B',
 'Etobicoke',
 'Cloverdale\n',

In [9]:
#formating the data from the list into a dataframe
def divide_chunks(l, n): 
      
    # looping till length l 
    for i in range(0, len(l), n):  
        yield l[i:i + n] 
  
# How many elements each 
# list should have 
n = 3
  
x = list(divide_chunks(table_text, n)) 
#print (x) 

df = pd.DataFrame(x, columns =['Postcode', 'Borough','Neighborhood'])
df.head()

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n


<b>Step 3: Here we go for data wrangling and cleaning once we get the data in a dataframe</b>

In [10]:
#data wrangling
df['Neighborhood'] = df['Neighborhood'].map(lambda x: str(x)[:-1])

In [11]:
df = df[df.Borough != 'Not assigned']
df.head(10)

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge
12,M1B,Scarborough,Malvern


In [12]:
df.shape

(211, 3)

In [13]:
#providing brough name to neighborhoods with name not assigned
is_NA  = df['Neighborhood'] == 'Not assigned'
#print(is_NA.head(5))
df['Boolean'] = is_NA
print(df.head(10))
for i in range(len(df)):
    if df.iloc[i,3] == True:
        df.iloc[i, 2] = df.iloc[i, 1]
df.head(10)       

   Postcode           Borough      Neighborhood  Boolean
2       M3A        North York         Parkwoods    False
3       M4A        North York  Victoria Village    False
4       M5A  Downtown Toronto      Harbourfront    False
5       M5A  Downtown Toronto       Regent Park    False
6       M6A        North York  Lawrence Heights    False
7       M6A        North York    Lawrence Manor    False
8       M7A      Queen's Park      Not assigned     True
10      M9A         Etobicoke  Islington Avenue    False
11      M1B       Scarborough             Rouge    False
12      M1B       Scarborough           Malvern    False


,Postcode,Borough,Neighborhood,Boolean
2,M3A,North York,Parkwoods,False
3,M4A,North York,Victoria Village,False
4,M5A,Downtown Toronto,Harbourfront,False
5,M5A,Downtown Toronto,Regent Park,False
6,M6A,North York,Lawrence Heights,False
7,M6A,North York,Lawrence Manor,False
8,M7A,Queen's Park,Queen's Park,True
10,M9A,Etobicoke,Islington Avenue,False
11,M1B,Scarborough,Rouge,False
12,M1B,Scarborough,Malvern,False


In [14]:
#clubbing common boroughs
print(df.head(20))
df1 = df.groupby(['Postcode','Borough'])['Neighborhood'].apply(''.join).reset_index()
df1.head(211)

   Postcode           Borough      Neighborhood  Boolean
2       M3A        North York         Parkwoods    False
3       M4A        North York  Victoria Village    False
4       M5A  Downtown Toronto      Harbourfront    False
5       M5A  Downtown Toronto       Regent Park    False
6       M6A        North York  Lawrence Heights    False
7       M6A        North York    Lawrence Manor    False
8       M7A      Queen's Park      Queen's Park     True
10      M9A         Etobicoke  Islington Avenue    False
11      M1B       Scarborough             Rouge    False
12      M1B       Scarborough           Malvern    False
14      M3B        North York   Don Mills North    False
15      M4B         East York  Woodbine Gardens    False
16      M4B         East York     Parkview Hill    False
17      M5B  Downtown Toronto           Ryerson    False
18      M5B  Downtown Toronto   Garden District    False
19      M6B        North York         Glencairn    False
22      M9B         Etobicoke  

,Postcode,Borough,Neighborhood
0,M1B,Scarborough,RougeMalvern
1,M1C,Scarborough,Highland CreekRouge HillPort Union
2,M1E,Scarborough,GuildwoodMorningsideWest Hill
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,East Birchmount ParkIonviewKennedy Park
7,M1L,Scarborough,ClairleaGolden MileOakridge
8,M1M,Scarborough,CliffcrestCliffsideScarborough Village West
9,M1N,Scarborough,Birch CliffCliffside West


<b>Step 4 : Here we will add the co-ordinates ( latitude & longitude) for each listed neighborhood</b>

In [15]:
!wget -O Geospatial_Coordinates.csv http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
longlat = pd.read_csv("Geospatial_Coordinates.csv")
longlat.head()

--2019-09-08 12:07:58--  http://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Resolving cocl.us (cocl.us)... 169.48.113.201
Connecting to cocl.us (cocl.us)|169.48.113.201|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv [following]
--2019-09-08 12:07:58--  https://cocl.us/Geospatial_data/Geospatial_Coordinates.csv
Connecting to cocl.us (cocl.us)|169.48.113.201|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2019-09-08 12:07:58--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.26.197
Connecting to ibm.box.com (ibm.box.com)|107.152.26.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
df1.shape

(103, 3)

In [17]:
longlat.shape

(103, 3)

In [18]:
longlat.head(103)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
5,M1J,43.744734,-79.239476
6,M1K,43.727929,-79.262029
7,M1L,43.711112,-79.284577
8,M1M,43.716316,-79.239476
9,M1N,43.692657,-79.264848


In [19]:
toronto = df1.join(longlat)
toronto.head()


,Postcode,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M1B,Scarborough,RougeMalvern,M1B,43.806686,-79.194353
1,M1C,Scarborough,Highland CreekRouge HillPort Union,M1C,43.784535,-79.160497
2,M1E,Scarborough,GuildwoodMorningsideWest Hill,M1E,43.763573,-79.188711
3,M1G,Scarborough,Woburn,M1G,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,M1H,43.773136,-79.239476


In [20]:
toronto.shape

(103, 6)